# Palmer Penguins

This notebook is inspired by an example [Radar chart with ggradar](https://r-graph-gallery.com/web-radar-chart-with-R.html).

In [1]:
%use dataframe
%useLatestDescriptors
%use lets-plot

In [2]:
LetsPlot.getInfo()

Lets-Plot Kotlin API v.4.12.0. Frontend: Notebook with dynamically loaded JS. Lets-Plot JS v.4.8.1.
Outputs: Web (HTML+JS), Kotlin Notebook (Swing), Static SVG (hidden)

In [3]:
fun rescaleInGroupDataFrame(df: DataFrame<*>, valueCol: String, groupCol: String, rescaledCol: String? = null): DataFrame<*> {
    val rescaledColName = if (rescaledCol == null) {
        "rescaled_${valueCol}"
    } else {
        rescaledCol
    }
    fun rescaleSubDataFrame(subDf: DataFrame<*>): DataFrame<*> {
        val minValue = subDf.minByOrNull(valueCol)?.let { it[valueCol] } as Double
        val maxValue = subDf.maxByOrNull(valueCol)?.let { it[valueCol] } as Double
        return subDf.add(rescaledColName) { (valueCol<Double>() - minValue) / (maxValue - minValue) }
    }
    return df.select(groupCol).distinct().map { v -> rescaleSubDataFrame(df.filter { groupCol<String>() == v[groupCol] }) }.concat()
}

fun getData(): DataFrame<*> {
    val df = DataFrame.readCSV("https://raw.githubusercontent.com/JetBrains/lets-plot-docs/master/data/penguins.csv")
        .dropNulls()
        .rename("bill_length_mm", "bill_depth_mm", "flipper_length_mm", "body_mass_g")
            .into("avg. bill length", "avg. bill depth", "avg. flipper length", "avg. body mass")
        .groupBy("species")
        .mean()
        .gather("avg. bill length", "avg. bill depth", "avg. flipper length", "avg. body mass")
            .into("variable", "value")
        .update { "value"<Double>() }
            .where { "variable"<String>() == "avg. body mass" }
            .with { it / 1000 }
        .add("units") {
            when ("variable"<String>()) {
                "avg. body mass" -> "kg"
                else -> "mm"
            }
        }
    return rescaleInGroupDataFrame(df, "value", "variable")
        .add("rescaled_value_pct") { floor(100 * "rescaled_value"<Double>()) }
        .convert { "rescaled_value_pct"<Double>() }.to<Int>()
        .sortBy("species")
}

In [4]:
val df = getData()
df.head(4)

species,variable,value,units,rescaled_value,rescaled_value_pct
Adelie,avg. bill length,38.823973,mm,0.000000,0
Adelie,avg. bill depth,18.347260,mm,0.978584,97
Adelie,avg. flipper length,190.102740,mm,0.000000,0
Adelie,avg. body mass,3.706164,kg,0.000000,0


In [5]:
val fontFamily = "roboto"
val axisColor = "lightgray"
val axisTextData = mapOf(
    "x" to List(3) { "avg. bill length" },
    "y" to listOf(0, 50, 100),
    "text" to listOf("0%", "50%", "100%")
)
val penguinsColors = mapOf(
    "Adelie" to "#ff5a5f",
    "Chinstrap" to "#ffb400",
    "Gentoo" to "#007a87"
)
val penguinsTooltips = layerTooltips().title("@species").line("@variable (@units): @value").format("@value", ".2~f")
val penguinsTheme = theme(
    text = elementText(family = fontFamily, size = 18),
    plotTitle = elementText(size = 28, hjust = .5, face = "bold"),
    axisTitle = "blank", axisTextY = "blank", axisLineX = "blank",
    axisTicks = elementLine(color = axisColor),
    panelGrid = elementLine(color = axisColor),
    panelInset = Pair(0, 100),
    tooltip = elementRect(),
    axisTooltip = "blank",
).legendPosition(1, 0).legendJustification(1, 0)

In [6]:
letsPlot(df.toMap()) +
    geomArea(position = positionIdentity, flat = true,
             size = 2.5, colorBy = "paint_a", fillBy = "paint_a", alpha = .2)
        { x = "variable"; y = "rescaled_value_pct"; paint_a = "species" } +
    geomPoint(size = 6, colorBy = "paint_a", tooltips = penguinsTooltips)
        { x = "variable"; y = "rescaled_value_pct"; paint_a = "species" } +
    geomText(data = axisTextData, hjust = 1, fontface = "bold", family = fontFamily, size = 10)
        { x = "x"; y = "y"; label = "text" } +
    scaleXDiscrete() +
    scaleManual("paint_a", name = "", values = penguinsColors) +
    coordPolar(ylim = Pair(-15, 100)) +
    ggsize(800, 600) +
    ggtitle("Penguins species") +
    penguinsTheme + flavorSolarizedLight()

<path d="M268.0 239.1103126122889 L273.6360984154138 239.6654198546821 L279.0556047294837 241.30940912184428 L284.0502503538541 243.9791028281981 L288.42809385821 247.57190614179 L292.0208971718019 251.9497496461459 L294.6905908781557 256.9443952705163 L296.3345801453179 262.3639015845862 L296.8896873877111 268.0 L296.3345801453179 273.6360984154138 L294.6905908781557 279.0556047294837 L292.0208971718019 284.05025035385415 L288.42809385821 288.42809385821 L284.0502503538541 292.0208971718019 L279.0556047294837 294.6905908781557 L273.63609841541387 296.3345801453179 L268.0 296.8896873877111 L262.36390158458613 296.3345801453179 L256.9443952705163 294.6905908781557 L251.9497496461459 292.0208971718019 L247.57190614179 288.42809385821 L243.9791028281981 284.05025035385415 L241.30940912184425 279.0556047294837 L239.6654198546821 273.63609841541387 L239.1103126122889 268.0 L239.6654198546821 262.3639015845862 L241.30940912184425 256.9443952705163 L243.9791028281981 251.9497496461459 L247.57190614179 247.57190614179 L251.94974964614588 243.9791028281981 L256.9443952705163 241.30940912184428 L262.36390158458613 239.6654198546821 L268.0 239.1103126122889 " stroke="rgb(211,211,211)" stroke-opacity="1.0" stroke-width="1.0" fill="none">
 
 <path d="M268.0 200.59072942867408 L274.60726393310296 200.91532349130284 L281.1508963026323 201.88597966092487 L287.56787835175606 203.49334998801208 L293.79641103546203 205.72195461763658 L299.77651017914377 208.55033086893593 L305.45058415899297 211.95123993246227 L310.76398854081924 215.89192919480797 L315.66555233582335 220.33444766417665 L320.108070805192 225.23601145918076 L324.0487600675377 230.54941584100703 L327.44966913106407 236.22348982085623 L330.2780453823634 242.20358896453797 L332.5066500119879 248.43212164824394 L334.1140203390751 254.8491036973677 L335.08467650869716 261.39273606689704 L335.4092705713259 268.0 L335.08467650869716 274.60726393310296 L334.1140203390751 281.1508963026323 L332.5066500119879 287.56787835175606 L330.2780453823634 293.79641103546203 L327.44966913106407 299.77651017914377 L324.0487600675377 305.4505841589929 L320.108070805192 310.76398854081924 L315.66555233582335 315.66555233582335 L310.76398854081924 320.108070805192 L305.4505841589929 324.0487600675377 L299.77651017914377 327.44966913106407 L293.79641103546203 330.2780453823634 L287.56787835175606 332.5066500119879 L281.1508963026323 334.1140203390751 L274.60726393310296 335.08467650869716 L268.0 335.4092705713259 L261.39273606689704 335.08467650869716 L254.8491036973677 334.1140203390751 L248.43212164824394 332.5066500119879 L242.20358896453797 330.2780453823634 L236.2234898208562 327.44966913106407 L230.54941584100706 324.0487600675377 L225.2360114591808 320.108070805192 L220.33444766417668 315.66555233582335 L215.89192919480797 310.76398854081924 L211.95123993246227 305.4505841589929 L208.5503308689359 299.77651017914377 L205.72195461763658 293.79641103546203 L203.49334998801208 287.56787835175606 L201.88597966092487 281.1508963026323 L200.91532349130284 274.60726393310296 L200.59072942867408 268.0 L200.91532349130284 261.39273606689704 L201.88597966092487 254.8491036973677 L203.49334998801208 248.43212164824394 L205.72195461763658 242.20358896453797 L208.5503308689359 236.22348982085623 L211.95123993246227 230.5494158410071 L215.89192919480797 225.23601145918076 L220.33444766417665 220.33444766417665 L225.23601145918076 215.89192919480797 L230.54941584100706 211.95123993246227 L236.2234898208562 208.55033086893593 L242.20358896453791 205.72195461763658 L248.43212164824394 203.49334998801208 L254.8491036973677 201.88597966092487 L261.39273606689704 200.91532349130284 L268.0 200.59072942867408 " stroke="rgb(211,211,211)" stroke-opacity="1.0" stroke-width="1.0" fill="none">
 
 <path d="M268.0 162.0711462450593 L278.38284332344756 162.58122262919022 L288.66569418985074 164.1065394671677 L298.7495231241881 166.63240712401898 L308.53721734144034 170.13450011342894 L317.93451599579737 174.57909136547073 L326